In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# Семинар – Деревья решений 

## 1. Построение и визуализация деревьев решений
Для нагляности в качестве простого примера, возьмем всем известные ирисы:

__150 объектов__  
__4 признака: __ 
- длина чашелистика (см)  
- ширина чашелистика (см)  
- длина лепестка (см)  
- ширина лепестка (см)  

__3 класса:__  
- Ирис щетинистый (Iris setosa)  
- Ирис виргинский (Iris virginica)  
- Ирис разноцветный (Iris versicolor)  

In [ ]:
# Загрузим данные
from sklearn.datasets import load_iris

iris = load_iris()

X = iris.data[:, 2:] # Возьмем только два признака: petal length и width
y = iris.target

target_names = iris.target_names
feature_names = iris.feature_names[2:]

Мы не будем делить выборку на трейн/тест, а для демонстрации просто построим решающее дерево на всех объектах

### Построение дерева решений
В sklearn за это отвечают `DecisionTreeClassifier` (для классификации) и `DecisionTreeRegressor` (для регрессии) из модуля `tree`

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Обучите свое первое дерево классификации с параметрами: `max_depth=2` и `random_state=42`, в качестве данных для обучения используйте X и y

In [ ]:
tree_clf = # <Ваш код здесь>

Мы ограничили глубину дерева (за это овечает параметр `max_depth`), подробнее о том, для чего мы это сделали поговорим позже.  
  
Кроме того, не забываем фиксировать `random_state` для воспроизводимости результатов

### Визуализация обученного дерева

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
# Отрисуем дерево
export_graphviz(tree_clf, feature_names=feature_names,
                class_names=target_names, 
                out_file='iris_tree.dot', filled=True)

Мы будем использовать https://dreampuf.github.io/GraphvizOnline/ для отрисовки. 

Однако, с помощью библиотеки pydot (pip install pydot) и например команды: 
`!dot -Tpng 'iris_tree.dot' -o 'iris_tree.png'` можно перевести .dot в изображение и вывести дерево прямо в ноутбуке.

По умолчанию в sklearn используется критерий Gini, но его можно поменять с помощью параметра `criterion`

__Напоминание:__
$$G_i = 1 - \sum_{k=1}^{n} {(p_{k})^2}$$
$$G_{split} = \frac {L}{N} \times {G_L} + \frac {R}{N} \times {G_R} \rightarrow min $$

где:
- $L$ - Количество элементов в левой ветке
- $R$ -Количество элементов в правой ветке
- $N$ - Количество элементов в узле  

__Подведем некоторые итоги:__
1. Дерево легко интерпретируется 
2. И прирост информации, и критерий Джини очевидно обощаются на многоклассовый случай
3. Дерево умеет предсказывать не только класс, но и вероятность отнесения к классу (с помощью метода `predict_proba`)

### Разделяющая поверхность
Так как у нас всего два признака, давайте проследим, как строится разделяющие границы

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    # Создадим точки по осям
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    # Создадим координатную матрицу 
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    #Согласно предсказаниям модели, закрасим нужные нам границы, предикт для каждой точки сделаем прямо внутри
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    # Сделаем функцию более унивесальной, чтобы применятьне только к ирисам
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    # Накидаем точек из Трейна
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa") # желтые (y) круги (o)
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor") # синие (b) квадраты (s)
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris-Virginica") # зеленые (g) треугольники (^)
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

plt.figure(figsize=(10, 4))
# Нарисуем границу предсказаний
plot_decision_boundary(tree_clf, X, y)

# Явно нарисуем линии границ, получившегося ранее дерева 
plt.plot([2.45, 2.45], [0, 3], "k-", linewidth=2)
plt.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.plot([4.95, 4.95], [0, 1.75], "k:", linewidth=2)
plt.plot([4.85, 4.85], [1.75, 3], "k:", linewidth=2)
plt.text(1.40, 1.0, "Depth=0", fontsize=15)
plt.text(3.2, 1.80, "Depth=1", fontsize=13)
plt.text(4.05, 0.5, "(Depth=2)", fontsize=11)

plt.show()

Мы не будем останавливаться подробно на коде, а оставим это в качестве дополнительных материалов по matplotlib

## 2. Переобучение и борьба с ним

In [ ]:
#Вернем все признаки в нашу выборку
iris = load_iris()
X = iris.data
target_names = iris.target_names
feature_names = iris.feature_names
y = iris.target

Постройте Решающее дерево с параметрами по умолчанию (random_state=42)
И отрисуйте его. 
На этот раз, мы не будем никак ограничивать наше дерево

In [ ]:
# Ваш код здесь

Так как при построении дерева используется принцип жадной максимизации, то дерево достаточно легко переобучить


### Объявляем борьбу с переобучением
Основные параметры класса [sklearn.tree.DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html):

- `max_depth` – максимальная глубина дерева
- `max_features` - максимальное число признаков, по которым ищется лучшее разбиение в дереве (это нужно потому, что при большом количестве признаков будет "дорого" искать лучшее (по критерию типа прироста информации) разбиение среди *всех* признаков)
- `min_samples_leaf` - минимальное число объектов в листе. У этого параметра есть понятная интерпретация: скажем, если он равен 5, то дерево будет порождать только те классифицирующие правила, которые верны как мимимум для 5 объектов
- `min_samples_split` - Минимальное колличество объектов для разбиения 

### Разберем еще один пример
Создадим игрушечный датасет make_moons (он действительно создает выборку два класса, по двум признкам в форме лун, которые входят одна в другу)

In [ ]:
from sklearn.datasets import make_moons
#Создадим луны
Xm, ym = make_moons(n_samples=100, noise=0.28, random_state=53)

#Обучим Первый классификатор без ограничений 
deep_tree_clf1 = DecisionTreeClassifier(random_state=42)
deep_tree_clf1.fit(Xm, ym)

#Обучим Второй классификатор ограничив число объектов в листе
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
deep_tree_clf2.fit(Xm, ym)

# Нарисуем decision_boundary для каждой и модели
plt.figure(figsize=(20, 4))
plt.subplot(121)
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
plt.title("No restrictions", fontsize=16)
plt.subplot(122)
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
plt.title("min_samples_leaf = {}".format(deep_tree_clf2.min_samples_leaf), fontsize=14)

plt.show()

Поэкспериментируйте в примере Выше с параметрами (Правое дерево) и посмотрите как изменяется разделяющая поверхность. 
Попробуйте разные значения для каждого из параметров, а так же их комбинации:
- `max_depth`
- `min_samples_leaf`
- `min_samples_split`


## 3. Чувствительность дерева к выборкам
Деревья решений очень чувствительны к положению данных в пространстве, так как строит разделяющую поверхность ортогонально относительно переменной. 

Рассмотрим еще один пример на игрушечных данных

In [ ]:
np.random.seed(6)
# Создадим набор случайных точек с двумя признаками:
Xs = np.random.rand(100, 2)*4 - 2
# Целевая переменная будет просто разделяться по одному признаку в 0
ys = (Xs[:, 0] > 0).astype(np.float32) * 2

# На основе созданной выборки, создадим еще одну, просто путем поворота 
angle = np.pi / 4
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

# Для каждой выборки обучим дерево решений   
tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_sr.fit(Xsr, ys)

# Сравним их
plt.figure(figsize=(20, 4))
plt.subplot(121)
plot_decision_boundary(tree_clf_s, Xs, ys, axes=[-2, 2, -2, 2], iris=False)
plt.subplot(122)
plot_decision_boundary(tree_clf_sr, Xsr, ys, axes=[-2, 2, -2, 2], iris=False)

plt.show()

В случае с повернутой выборкой, рещающая граница стала более сложной. 
Подумайте, какая модель справилась бы лучше с такой природой данных

## Часть 4 - Регрессионые решающие деревья

 При прогнозировании вещественного признака идея построения дерева остается та же, но меняется критерий качества: 
 
 - Дисперсия вокруг среднего: $$\Large D = \frac{1}{\ell} \sum\limits_{i =1}^{\ell} (y_i - \frac{1}{\ell} \sum\limits_{i =1}^{\ell} y_i)^2, $$
 где $\ell$ – число объектов в листе, $y_i$ – значения целевого признака. Попросту говоря, минимизируя дисперсию вокруг среднего, мы ищем признаки, разбивающие выборку таким образом, что значения целевого признака в каждом листе примерно равны.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

Создадим выборку на основе зашумленной функции: 
$$y = 4(x-0,5)^2$$

In [ ]:
np.random.seed(42)
# Сгинерируем вектор X состоящий из 200 точек 
m = 200
X = np.random.rand(m, 1)

In [ ]:
#Создадим Y на основе нашей функции 
y = # Ваш код здесь
# Добавим нормальный шум вокруг данных (чтобы шум был не такой сильный, разделим его на 10), поможет np.random.randn(200,1)
y = # Ваш код здесь

### Обучим решающее дерево

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

### Визуализация и разбор процесса построения

In [ ]:
# Обучим дерево решений ограничим его глубинной 2 (random_state=42)
tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg1.fit(X, y)

# Обучим еще одно дерево, но теперь поставим ограничение на глбину 3 (random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg2.fit(X, y)

Визуализируем каждое из деревьев:

In [ ]:
def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(20, 4))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)


plt.show()

### Проблема переобучения
Регрессионное дерево имеет ту же природу, что и дерево классикации и строится жадно, поэтому имеет те же проблемы с переобучением 

In [ ]:
# Сравним, как будет выглядеть дерево без ограничений 
tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg1.fit(X, y)

# и дерево в которое мы внесем ограничения
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg2.fit(X, y)

# Посмотрим, как каждая из моделей справляется с новыми данными
x1 = np.linspace(0, 1, 500).reshape(-1, 1)
# Сделаем предсказания: 
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

# Отрисуем результаты предсказаний
plt.figure(figsize=(20, 4))

plt.subplot(121)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("No restrictions", fontsize=14)

plt.subplot(122)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf={}".format(tree_reg2.min_samples_leaf), fontsize=14)

plt.show()

## Заключение. Плюсы и минусы деревьев решений

**Плюсы:**
 - Деревья решений легко визуализировать и интерпретировать;
 - Быстрые процессы обучения и прогнозирования;
 - Малое число параметров модели.
 
**Минусы:**
 - Деревья очень чувствительны к шумам во входных данных, вся модель может кардинально измениться, если немного изменится обучающая выборка, поэтому и правила классификации могут сильно изменяться;
 - Разделяющая граница, построенная деревом решений, имеет свои ограничения (состоит из гиперплоскостей, перпендикулярных какой-то из координатной оси);
 - Необходимость отсекать ветви дерева (pruning) или устанавливать минимальное число элементов в листьях дерева или максимальную глубину дерева для борьбы с переобучением;
 - Жадный поиск признака с максимальным приростом информации не гарантируют нахождения глобально оптимального дерева;
 - Сложно поддерживаются пропуски в данных.

## Бонус-трек: Интуиция Ансамблирования

<img src='bias_variance.png'>

### Феномен: «Мудрость толпы»

В 1906 году британский ученый сэр Фрэнсис Гальтон посещал выставку достижений животноводства, где случайно провел крайне важное наблюдение. 

На выставке проводился конкурс, в рамках которого всем желающим предлагалось на глаз угадать точный вес забитого быка. Побеждал тот, кто называл самое близкое к истинному значение. 

Полагая, что справиться с подобной задачей под силу только профессионалу, и чтобы доказать некомпетентность толпы, Гальтон посчитал среднее значение из почти восьми сотен догадок посетителей ярмарки. К удивлению ученого, толпа ошиблась меньше, чем на килограмм. 

### Теорема Кондорсе о присяжных
Если каждый член жюри присяжных имеет независимое мнение, и если вероятность правильного решения члена жюри больше 0.5, то тогда вероятность правильного решения присяжных в целом возрастает с увеличением количества членов жюри, и стремиться к единице. Если же вероятность быть правым у каждого из членов жюри меньше 0.5, то вероятность принятия правильного решения присяжными в целом монотонно уменьшается и стремится к нулю с увеличением количества присяжных.

$$\mu = \sum_{i=m}^{N}{}C^i_N p^i (1-p)^{N-i}$$

При $ p > 0,5 $, то $ \mu > p $  
  
Если дополнительно добавить условие $N \rightarrow \infty $, то $ \mu  \rightarrow 1 $

### Мы сделаем идеальный алгоритм?
_Спойлер:_ Нет

<img src='galton.png'>

## Резюме

- Обучение решающего дерева
- Подбор оптимальных параметров для дерева
- Дерево решений для задачи регрессии
- Решение задачи классификация рукописных цифр 
- Оценка качества при многокласовой классификации


**Ноутбук составлен по мотивам:**
- [Открытый курс ODS по машинному обучению](https://habr.com/company/ods/blog/322626/)
- [Блог](https://alexanderdyakonov.wordpress.com/2016/11/14/случайный-лес-random-forest/) Александра Дьяконова
- [Курс](https://github.com/esokolov/ml-course-hse) Евгения Соколова по машинному обучению (материалы на GitHub)
- Прикладное машинное обучение с помощью Scikit-Learn и TensorFlow // Орельен Жерон